In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Ejercicio 38

Para cada categoría, calcular cuál es el caracter predominante en la misma. El caracter predominante en una categoria esta dado por el caracter que es dominante en mayor número de apps, por ej para aaaaaaax, xxb, xxs el caracter dominante de ap1 es a, pero para las otras 2 x, por lo que x es el caracter dominante en la categoría. Si no hay caracter dominante, es el primer caracter. (⭐⭐)

Leo los csv usando únicamente las columnas que serán útiles para este ejercicio.

Como analizaré caracteres en los nombres de las aplicaciones por categorías, solo tendré en cuenta las columnas App y Category.

In [3]:
apps_by_category = pd.read_csv('/content/drive/MyDrive/Organización de Datos/googleplaystore.csv', usecols=['App', 'Category'])

Hay apps que figuran repetidas.

Algunas figuran dos veces con la misma categoría, lo que me haría contar caracteres dominantes de más.

Otras pertenecen a distintas categorías. Voy a suponer que cada app debería pertenecer a una categoría únicamente, por ende considero la situación mencionada un error de los datos y eliminaré los duplicados para no tener conflictos. Esto causará la eliminación de algunas app según categoría. Por ejemolo, si una aplicación aparece como perteneciente a la categoríaX y a la categoríaY, solo quedará como perteneciente a una y se "perderá" que pertenecía a la otra. Obviamente, esto modificará los resultados finales, pero lo considero una mejor alternativa a considerar que una app pertenece a dos categorías distintas.

Además, hay una categoría con nombre sin sentido (1.9), la eliminaré también.

In [4]:
apps_by_category[apps_by_category.App == 'Box'] #ejemplo de repetición de una app (misma categoría) (no necesario resolución)

,App,Category
204,Box,BUSINESS
236,Box,BUSINESS
265,Box,BUSINESS


In [5]:
apps_by_category[apps_by_category.App == 'Coloring book moana'] #ejemplo de repetición de una app (con distintas categorías) (no necesario resolución)

,App,Category
1,Coloring book moana,ART_AND_DESIGN
2033,Coloring book moana,FAMILY


In [6]:
apps_by_category.Category.unique() #categorías -> 1.9 no tiene sentido (no necesario resolución)

array(['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY',
       'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION',
       'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE',
       'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME',
       'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL',
       'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL',
       'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER',
       'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION',
       '1.9'], dtype=object)

In [7]:
apps_by_category = apps_by_category.drop_duplicates(subset='App')
apps_by_category = apps_by_category[apps_by_category.Category != '1.9']

De los nombres de las aplicaciones, reemplazo espacios así no aparecen como dominantes, no es necesario pero así no queda "feo" (podría hacerse también con signos de puntuación)

In [8]:
def caracter_dominante(app):
    app = app.replace(" ","")
    app = app.lower()

    max_apariciones=0
    primer_caracter=app[0]
    caracter_dom=primer_caracter

    for caracter in app:
        apariciones = app.count(caracter)
        if apariciones > max_apariciones:
          max_apariciones = apariciones
          caracter_dom = caracter
        if (caracter != caracter_dom) & (apariciones == max_apariciones):
          caracter_dom = primer_caracter


    return caracter_dom

In [9]:
apps_by_category["Dominant Char"] = apps_by_category.App.map(caracter_dominante)

In [10]:
apps_by_category.head(3) #tengo el caracter dominante por cada app (no necesario resolución)

,App,Category,Dominant Char
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,o
1,Coloring book moana,ART_AND_DESIGN,o
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,e


In [11]:
apps_by_category[['Category','Dominant Char']].value_counts() #(no necesario resolución)

Category        Dominant Char
FAMILY          e                281
                a                274
TOOLS           e                164
GAME            e                136
FAMILY          c                125
                                ... 
FOOD_AND_DRINK  n                  1
                l                  1
SOCIAL          j                  1
FOOD_AND_DRINK  k                  1
ART_AND_DESIGN  3                  1
Length: 796, dtype: int64

Haciendo value counts obtengo, por categoría, cuántas veces un caracter es dominante en una app, en orden descendente. Si esto lo agrupo por categoría y tomo el primer elemento (se toma por cada categoría), tendré lo que se pide.

In [12]:
apps_by_category[['Category','Dominant Char']].value_counts().groupby("Category").head(1).reset_index()[['Category','Dominant Char']]

,Category,Dominant Char
0,FAMILY,e
1,TOOLS,e
2,GAME,e
3,BUSINESS,e
4,PERSONALIZATION,e
5,PRODUCTIVITY,e
6,MEDICAL,e
7,PHOTOGRAPHY,o
8,FINANCE,e
9,LIFESTYLE,a


Como comentario:

Otra manera es usar el describe, que directamente me da la solución (top).

Lo pongo como "segunda opción" porque no sé realmente como se calcula top en el describe, así que desconozco si es mejor que mi solución, lo cual no creo ya que además estoy haciendo count y unique en el describe, operaciones innecesarias.

¡¡¡¡¡¡¡¡¡¡¡Considerar lo hecho anteriormente como solución del ejercicio!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [13]:
dominant_chars_by_category2 = apps_by_category[['Category', 'Dominant Char']].groupby("Category").describe().reset_index()

In [14]:
dominant_chars_by_category2.head(3) #tengo el caracter dominante de cada categoría (top) y su frequencia (freq) (no necesario)

Category Dominant Char                
                             count unique top freq
0     ART_AND_DESIGN            64     18   e   13
1  AUTO_AND_VEHICLES            85     18   e   15
2             BEAUTY            53     14   e   13

In [15]:
dominant_chars_by_category2.columns=["Category", "Apps", "Characters", "Dominant Char", "Freq"]
dominant_chars_by_category2 = dominant_chars_by_category2[["Category", "Dominant Char"]]

In [16]:
dominant_chars_by_category2 #se obtiene mismo resultado :)

,Category,Dominant Char
0,ART_AND_DESIGN,e
1,AUTO_AND_VEHICLES,e
2,BEAUTY,e
3,BOOKS_AND_REFERENCE,e
4,BUSINESS,e
5,COMICS,a
6,COMMUNICATION,e
7,DATING,e
8,EDUCATION,e
9,ENTERTAINMENT,t
